In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=len(desired_samples), figsize=(15, 10))

# Setting sample rate and STFT parameters
sample_rate = 16000
n_fft_value = 512
hop_length_value = 256  # Adjust as needed for time resolution

for i,  (x, y) in enumerate(desired_samples):
    x = x.squeeze()
    y = y.squeeze()

    x_D = librosa.amplitude_to_db(np.abs(librosa.stft(x, n_fft=n_fft_value, hop_length=hop_length_value)), ref=np.max)
    y_D = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=n_fft_value, hop_length=hop_length_value)), ref=np.max)
    diff_D = y_D - x_D

    # Visualizing the Input
    x_img = librosa.display.specshow(x_D, y_axis='log', x_axis='time', sr=sample_rate, ax=axs[0, i], cmap='hot')
    axs[0, i].set_title(f'Random Sample {i + 1} (Input)')
    fig.colorbar(x_img, ax=axs[0, i], format='%+2.0f dB')

    # Visualizing the Target
    y_img = librosa.display.specshow(y_D, y_axis='log', x_axis='time', sr=sample_rate, ax=axs[1, i], cmap='hot')
    axs[1, i].set_title(f'Random Sample {i + 1} (Target)')
    fig.colorbar(y_img, ax=axs[1, i], format='%+2.0f dB')
    
    # Visualizing the Difference
    diff_img = librosa.display.specshow(diff_D, y_axis='log', x_axis='time', sr=sample_rate, ax=axs[2, i], cmap='coolwarm')
    axs[2, i].set_title(f'Random Sample {i + 1} (Difference)')
    fig.colorbar(diff_img, ax=axs[2, i], format='%+2.0f dB')

plt.tight_layout()
plt.show()

In [ ]:
# Librosa Feature Extraction

def extract_features_librosa(audio, sample_rate) -> dict:
    """
    Extract audio features from a waveform signal with Librosa.
    """
    
    audio = np.squeeze(audio)

    # Compute RMS energy for loudness
    rms = librosa.feature.rms(y=audio).squeeze()

    # Spectral centroid
    centroid_val = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate))

    contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    
    flatness = librosa.feature.spectral_flatness(y=audio)

    # Return as a dictionary
    features = {
        'rms': np.mean(rms),
        'centroid': centroid_val,
        'contrast': np.mean(contrast),
        'flatness': np.mean(flatness),
    }
    return features

In [ ]:
files = ["springset_x_train.json", "springset_y_train.json", "springset_x_test.json", "springset_y_test.json"]

summary_df = pd.DataFrame()

for file in files:
    file_path = os.path.join(RESULTS_DIR, file)
    df = pd.read_json(file_path, lines=True)

    # Drop the index column(s)
    df = df.drop(columns=[col for col in df.columns if 'idx' in col])
    
    # Compute mean for each column and round to 3 decimal places
    mean = df.mean().round(3)

    # Assign the file name to a new column 'file'
    mean['file'] = file
    
    # Append the mean values to the summary dataframe
    summary_df = pd.concat([summary_df, pd.DataFrame(mean).T])

# Set 'file' as the index of the summary dataframe
summary_df.set_index('file', inplace=True)

summary_df

In [ ]:
# Define the notes and their positions relative to A4
notes = {
    'guitar': {'E2': -31, 'A2': -26, 'D3': -21, 'G3': -16, 'B3': -11, 'E4': -6},
    'bass': {'E1': -40, 'A1': -35, 'D2': -31, 'G2': -26}
}

# Constants for frequency calculation
f_0 = 440  # frequency of A4 in Hz
a = 2 ** (1/12)  # twelfth root of 2

# Calculate frequencies
for instrument in notes:
    for note in notes[instrument]:
        f = f_0 * (a ** notes[instrument][note])
        notes[instrument][note] = f

# Plot notes and their frequencies
fig, ax = plt.subplots()
for instrument in notes:
    names = list(notes[instrument].keys())
    frequencies = list(notes[instrument].values())
    ax.scatter(names, frequencies, label=instrument)
    for i, txt in enumerate(frequencies):
        ax.annotate(f'{txt:.2f} Hz', (names[i], frequencies[i]))

ax.set_xlabel('Note')
ax.set_ylabel('Frequency (Hz)')
ax.legend()
plt.grid()
plt.show()

In [ ]:
import seaborn as sns
import pandas as pd

# Convert the dictionary of notes to a DataFrame
df = pd.DataFrame(notes).T

# Plot the heatmap
plt.figure(figsize=(8,4))
sns.heatmap(df, annot=True, fmt=".2f", cmap='viridis')
plt.xlabel('Note')
plt.ylabel('Instrument')
plt.title('Frequencies of Guitar and Bass Notes')
plt.show()

In [ ]:
# Initialize list to hold individual summary dataframes
summary_dfs = []

for file in files:
    file_path = os.path.join(RESULTS_DIR, file)
    df = pd.read_json(file_path, lines=True)

    # Drop the index column(s)
    df = df.drop(columns=[col for col in df.columns if 'idx' in col])
    
    # Compute mean and standard deviation for each column and round to 3 decimal places
    mean = df.mean().round(3)
    std = df.std().round(3)
    
    # Convert the series to dataframes, transpose, and assign the file name to the index
    mean = pd.DataFrame(mean).T.assign(file=file, stat='mean')
    std = pd.DataFrame(std).T.assign(file=file, stat='std')

    # Append the dataframes to the list
    summary_dfs.extend([mean, std])

# Concatenate all the dataframes in the list into a single dataframe
summary_df = pd.concat(summary_dfs)

# Set 'file' and 'stat' as the multi-index of the summary dataframe
summary_df.set_index(['file', 'stat'], inplace=True)

# Plot two separate heatmaps for mean and standard deviation
for stat in ['mean', 'std']:
    plt.figure(figsize=(8, 4))
    sns.heatmap(summary_df.xs(stat, level='stat'), annot=True, cmap='viridis')
    plt.title(f'Heatmap of {stat} values')
    plt.show()
